# Table of Contents
 <p><div class="lev2"><a href="#Sotfmax-Regressions"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Sotfmax Regressions</a></div><div class="lev2"><a href="#implementing-the-regression"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>implementing the regression</a></div><div class="lev2"><a href="#Training"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Training</a></div><div class="lev2"><a href="#evaluating-our-model"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>evaluating our model</a></div><div class="lev2"><a href="#backpropagation"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>backpropagation</a></div><div class="lev3"><a href="#computational-graphs"><span class="toc-item-num">0.5.1&nbsp;&nbsp;</span>computational graphs</a></div><div class="lev3"><a href="#derivatives-on-computational-graphs"><span class="toc-item-num">0.5.2&nbsp;&nbsp;</span>derivatives on computational graphs</a></div><div class="lev3"><a href="#factoring-paths"><span class="toc-item-num">0.5.3&nbsp;&nbsp;</span>factoring paths</a></div><div class="lev2"><a href="#分からないこと"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>分からないこと</a></div>

MNIST(mixed national institude standards and technology) dataset : 手書きの数字データセット

Softmax Regression:

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TensorFlowEstimator : 

one-hot vectors : ある次元だけ1で他は0のベクトル（0-9の範囲で3なら3次元めだけ1)



## Sotfmax Regressions
入力x、あるクラスiである証拠は

$\rm{evidence}_{i} = \sum_{j} W_{ij}x_{j} + b_{i} $

証拠から確率に変換するソフトマックスは正規化された指数関数で表される

$y=\rm{softmax}(\rm{evidence})$

$\rm{softmax}(x)_{i}=\frac{\exp(x_{i}}{\sum_{j}\exp(x_{j})}$

これはポテンシャルエネルギーと確率の関係に対応していそう。

$y=\rm{softmax}(Wx+b)$

## implementing the regression

In [2]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 784])

xは"placeholder"と呼ばれ、計算を実行するときに値が入る。Noneはどんな長さでもいいことを表す。

In [3]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Variablesは計算中に値を変えることができるもので、パラメーターはVariableで表す。

In [4]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

softmaxを計算するには、単にベクトルをsoftmax関数に渡すだけ。

## Training
学習するにはコストを定義する必要がある。よく使われるのは、"Cross-entropy"

$H_{y'}(y)=-\sum_{i} y'_{i} \log (y_{i})$

y: 予測した確率分布  
y': 真の分布

真の分布は学習データから求めるの？  
なんで分布を一致させるだけでいい感じに予測できるの？



In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1] ))

さっきはWが行列だったので、matmulが必要だったけど、yはベクトルなので\*でOK。yはテンソルで1次元目はデータのインデックスなので2次元目について平均を計算する。

"backpropagation algorithm"を使って自動的にコストが小さくなるように計算される。

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [9]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x : batch_xs, y_:batch_ys})

## evaluating our model


In [10]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9179


## backpropagation

色々な分野で使われいて、応用と関係ない一般的な名前は"reverse-mode differentiation"

微分を速く計算するテクニック。

### computational graphs
$e = (a+b) (b+1)$を

$c = a+b \\
d = b+1\\
e = c d$

としてグラフで表す。

### derivatives on computational graphs
cがaの関数のとき、aを少しかえたとき、cはどうなるのか？これがcのaについての偏微分。

a = 2, b=1のとき、

$\frac{\partial c}{\partial a} = 1\\
\frac{\partial c}{\partial b} = 1\\
\frac{\partial d}{\partial b} = 1\\
\frac{\partial e}{\partial c} = 2\\
\frac{\partial e}{\partial d} = 3\\
$

と計算できる。

eのa,bについての偏微分も足し合わせればよい。

aが速度1でかわるとcも速度1でかわる。それはeが速度2でかわることになる。なので、 eはaに対して1\*2でかわる。

bについては、bが1でかわるとc、dがそれぞれ1でかわる。c,dがそれぞれ1でかわるとeはそれぞれ2,3でかわるので、すべてたしあわせて、1\*2+1\*3でかわる。

### factoring paths
パスが複数あると、単純に足し合わせると複雑なグラフになると足し算の項が指数的に増えてしまうという問題がある。->各ノードでかけることをにする

$\frac{\partial Z}{\partial X} = (\alpha + \beta + \gamma) (\delta + \epsilon + \xi)$

これが"forward-mode differentiation"と"reverse-mode differentiation"のもとになっているもの。

この計算のしかたに従って、ルートノードから順に$\frac{\partial Z}{\partial}$をかけていくのがバックプロパゲーション

末端ノードから計算するとすべての末端ノードからの計算をする必要があるが、ルートから計算すると一度にできる。

ニューラルネットワークにおいて、どのパラメーターをかえるとコストが小さくなるのかを見つけるために、微分を計算する必要があるので、このときに使われる。Gradient descentなどで最小値を探す。

## 分からないこと
具体的にどうやってパラメーターを更新しているのか？（バックプロパゲーションで偏微分を計算して探すっぽいことは分かった）